In [1]:
from BinanceTrade import *
from binance.client import Client
import pandas as pd
import numpy as np
import requests
import datetime
import yaml

In [2]:
with open('/Users/ho/Documents/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']

client = Client(api_key=APP_KEY, api_secret=APP_SECRET)

In [3]:
ticker = "BTCUSDT"
interval='1d'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

balance = 1000.0

df = get_ohlcv(client)

In [4]:
# # 레버리지 변경
# client.futures_change_leverage(
#     symbol="BTCUSDT",
#     leverage=3,
# )

# # 선물 마진 타입 변경
# client.futures_change_margin_type(
#     symbol="BTCUSDT",
#     marginType="ISOLATED",  # ISOLATED, CROSSED
# )

# # 선물 잔고 조회
# account = client.futures_account()
# for i in account['assets']:
#     if i['asset'] == 'USDT':
#         print(i)
# for i in account['positions']:
#     if i['symbol'] == 'BTCUSDT':
#         print(i)

# # 주문 생성
# try:
#     client.futures_create_order(
#         symbol="BTCUSDT",  # 주문 종목
#         timeInForce='GTC',
#         type="LIMIT",   # LIMIT, MARKET
#         side='BUY',     # BUY, SELL
#         price=20000,     # 주문 희망가
#         quantity=1     # 주문 수량
#     )
# except Exception as e:
#     print(e)

# # 모든 주문 취소
# client.futures_cancel_all_open_orders(
#     symbol="BTCUSDT"
# )

In [6]:
import pandas as pd

# CSV 파일에서 주식 가격 데이터를 로드합니다. 파일 경로를 적절하게 수정하세요.
data = get_ohlcv(client,limit=1000)

# RSI 계산
def calculate_rsi(data, period=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

data['RSI'] = calculate_rsi(data)

# MFI 계산
def calculate_mfi(data, period=14):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    raw_money_flow = typical_price * data['Volume']
    
    money_flow_positive = raw_money_flow.where(data['Close'] > data['Close'].shift(1), 0)
    money_flow_negative = raw_money_flow.where(data['Close'] < data['Close'].shift(1), 0)
    
    money_ratio = money_flow_positive.rolling(window=period).sum() / money_flow_negative.rolling(window=period).sum()
    mfi = 100 - (100 / (1 + money_ratio))
    
    return mfi

data['MFI'] = calculate_mfi(data)

# RMF 계산
def calculate_rmf(data):
    data['RMF'] = (data['RSI'] + data['MFI']) / 2
    return data

data = calculate_rmf(data)

# RMF 값을 출력하거나 저장합니다.
print(data['RMF'])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
995    52.873153
996    58.854001
997    56.143262
998    53.219452
999    61.702918
Name: RMF, Length: 1000, dtype: float64


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# CSV 파일에서 주식 가격 데이터를 로드합니다. 파일 경로를 적절하게 수정하세요.
data = get_ohlcv(client,limit=1000)

# VWMA 계산 함수
def calculate_vwma(data, period=14):
    data['Weighted_Price'] = data['Close'] * data['Volume']
    data['VWMA'] = data['Weighted_Price'].rolling(window=period).sum() / data['Volume'].rolling(window=period).sum()
    return data

# VWMA 계산
data = calculate_vwma(data)

# 상단 및 하단 밴드 계산
std_dev = 2  # 표준 편차 값 (상단 및 하단 밴드의 폭을 결정)
data['Upper_Band'] = data['VWMA'] + (std_dev * data['VWMA'].rolling(window=14).std())
data['Lower_Band'] = data['VWMA'] - (std_dev * data['VWMA'].rolling(window=14).std())

# # 그래프 그리기
# plt.figure(figsize=(12, 6))
# plt.plot(data.index, data['Close'], label='Stock Price', color='blue')
# plt.plot(data.index, data['VWMA'], label='VWMA', color='green')
# plt.fill_between(data.index, data['Upper_Band'], data['Lower_Band'], color='yellow', alpha=0.5, label='Band')
# plt.title('VWMA Band Graph')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.legend()
# plt.grid(True)
# plt.show()
data

,Time,Open,High,Low,Close,Volume,Weighted_Price,VWMA,Upper_Band,Lower_Band
0,2020-12-23,23810.79,24100.00,22600.00,23232.76,119047.259733,2.765796e+09,NaN,NaN,NaN
1,2020-12-24,23232.39,23794.43,22703.42,23729.20,69013.834252,1.637643e+09,NaN,NaN,NaN
2,2020-12-25,23728.99,24789.86,23433.60,24712.47,79519.943569,1.965134e+09,NaN,NaN,NaN
3,2020-12-26,24712.47,26867.03,24500.00,26493.39,97806.513386,2.591226e+09,NaN,NaN,NaN
4,2020-12-27,26493.40,28422.00,25700.00,26281.66,148455.586214,3.901659e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,1.016700e+09,25895.269042,26334.816391,25455.721693
996,2023-09-15,26522.73,26888.00,26224.00,26600.00,26227.293690,6.976460e+08,25952.925103,26415.425702,25490.424503
997,2023-09-16,26599.99,26777.00,26445.00,26559.67,13960.933510,3.707978e+08,25978.615203,26447.688930,25509.541475
998,2023-09-17,26559.67,26623.25,26399.00,26527.51,12998.102770,3.448073e+08,25997.831266,26461.983724,25533.678809


In [22]:
import pandas as pd

# CSV 파일에서 주식 가격 데이터를 로드합니다. 파일 경로를 적절하게 수정하세요.
data = get_ohlcv(client,limit=1000)

# RSI 계산
def calculate_rsi(data, period=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# MFI 계산
def calculate_mfi(data, period=14):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    raw_money_flow = typical_price * data['Volume']
    
    money_flow_positive = raw_money_flow.where(data['Close'] > data['Close'].shift(1), 0)
    money_flow_negative = raw_money_flow.where(data['Close'] < data['Close'].shift(1), 0)
    
    money_ratio = money_flow_positive.rolling(window=period).sum() / money_flow_negative.rolling(window=period).sum()
    mfi = 100 - (100 / (1 + money_ratio))
    
    return mfi

# RMF 계산
def calculate_rmf(data):
    return (data['RSI'] + data['MFI']) / 2

# VWMA 계산 함수
def calculate_vwma(data, period=14):
    data['Weighted_Price'] = data['Close'] * data['Volume']
    return data['Weighted_Price'].rolling(window=period).sum() / data['Volume'].rolling(window=period).sum()

data['RSI'] = calculate_rsi(data)
data['MFI'] = calculate_mfi(data)
data['RMF'] = calculate_rmf(data)
data['VWMA'] = calculate_vwma(data)

# 상단 및 하단 밴드 계산
std_dev = 2  # 표준 편차 값 (상단 및 하단 밴드의 폭을 결정)
data['Upper_Band'] = data['VWMA'] + (std_dev * data['VWMA'].rolling(window=14).std())
data['Lower_Band'] = data['VWMA'] - (std_dev * data['VWMA'].rolling(window=14).std())

data

,Time,Open,High,Low,Close,Volume,RSI,MFI,RMF,Weighted_Price,VWMA,Upper_Band,Lower_Band
0,2020-12-23,23810.79,24100.00,22600.00,23232.76,119047.259733,NaN,NaN,NaN,2.765796e+09,NaN,NaN,NaN
1,2020-12-24,23232.39,23794.43,22703.42,23729.20,69013.834252,NaN,NaN,NaN,1.637643e+09,NaN,NaN,NaN
2,2020-12-25,23728.99,24789.86,23433.60,24712.47,79519.943569,NaN,NaN,NaN,1.965134e+09,NaN,NaN,NaN
3,2020-12-26,24712.47,26867.03,24500.00,26493.39,97806.513386,NaN,NaN,NaN,2.591226e+09,NaN,NaN,NaN
4,2020-12-27,26493.40,28422.00,25700.00,26281.66,148455.586214,NaN,NaN,NaN,3.901659e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,58.407519,47.338786,52.873153,1.016700e+09,25895.269042,26334.816391,25455.721693
996,2023-09-15,26522.73,26888.00,26224.00,26600.00,26227.293690,61.682092,56.025911,58.854001,6.976460e+08,25952.925103,26415.425702,25490.424503
997,2023-09-16,26599.99,26777.00,26445.00,26559.67,13960.933510,60.214605,52.071919,56.143262,3.707978e+08,25978.615203,26447.688930,25509.541475
998,2023-09-17,26559.67,26623.25,26399.00,26527.51,12998.102770,58.406473,48.032432,53.219452,3.448073e+08,25997.831266,26461.983724,25533.678809


In [23]:
# 코인 설정
ticker = "ETHUSDT"
interval='1h'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000
df = get_ohlcv(client,ticker,interval,limit)

# 초기 설정
start_balance =10000.0 # 시작 자산
cash = start_balance
fee = 0.0005
rate = 0.1

# 전략 설정
bt = heikinashi_upswing_strategy(df,13,21)

bt[['현금', '보유수량', '총 자산', '수익률']] = 0.0
quantity = 0
bt.loc[0, '현금'] = start_balance
bt.loc[0, '총 자산'] = start_balance
for i in range(1, len(bt)):
    price = df.loc[i, 'Close']
    if bt.loc[i, 'Action'] == 'Buy':
        buy_quantity = cash*rate / price * (1-fee)  # 매수수량 = 현금*비율 / 현재가*(1-수수료)
        quantity += buy_quantity    # 수량 업데이트
        cash -= cash*rate   # 현금 업데이트
    elif bt.loc[i, 'Action'] == 'Sell':
        sell_quantity = quantity*rate  # 매도수량 = 보유수량*비율
        quantity -= sell_quantity   # 수량 업데이트 
        cash += sell_quantity * price * (1-fee) # 현금 = 기존현금 + 매도수량 * 현재가 * (1-수수료)
    bt.loc[i, '보유수량'] = quantity
    bt.loc[i, '현금'] = cash
    bt.loc[i, '총 자산'] = cash + (quantity*price)
    bt.loc[i, '수익률'] = (bt.loc[i,'총 자산'] - start_balance) / start_balance
bt['수익률'] *= 100

print("매수 횟수:", len(bt[bt['Action']=='Buy']), "번")
print("매도 횟수:", len(bt[bt['Action']=='Sell']), "번")
print("최고 수익률:", round(max(bt['수익률']),3), "%")
print("MDD:", round(min(bt['수익률']),3), "%")
print("최종 수익률:", round(bt['수익률'].iloc[-1],3), "%")

매수 횟수: 19 번
매도 횟수: 16 번
최고 수익률: 0.077 %
MDD: -8.132 %
최종 수익률: -4.794 %


In [39]:
# 코인 설정
ticker = "XRPUSDT"
interval='15m'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000
df = get_ohlcv(client,ticker,interval,limit)

# 초기 설정
start_balance =10000.0 # 시작 자산
cash = start_balance
fee = 0.0005
rate = 0.1

# 매매 조건에 따라 Action 컬럼 생성 함수
def create_action_column(data, rmf_threshold, dca_levels):
    actions = []
    dca_counter = 0  # DCA 수행 횟수

    for index, row in data.iterrows():
        rmf = row['RMF']
        vwma = row['VWMA']

        if vwma < rmf_threshold and rmf < rmf_threshold:
            if dca_counter < len(dca_levels):
                dca_counter += 1
                actions.append('Buy')
            else:
                actions.append('Stay')
        elif vwma > rmf_threshold and rmf > rmf_threshold:
            actions.append('Sell')
        else:
            actions.append('Stay')
    
    data['Action'] = actions
    return data

# RMF 임계값 및 DCA 레벨 설정
rmf_threshold = 30  # RMF가 이 임계값 아래인 경우 매수
dca_levels = [0.1, 0.2, 0.3]  # DCA 레벨 (첫 번째 매수는 자본의 10%, 두 번째는 20%, 세 번째는 30%)

# Action 컬럼 생성
data = create_action_column(data, rmf_threshold, dca_levels)

# 결과 출력
print(data['Action'].unique())

['Stay' 'Sell']


In [3]:
def srp_strategy(data, rmf_threshold, dca_levels=[0.1,0.2,0.5], std_dev=2):
    """
    SRP 전략
    매수 조건: VWMA 밴드 하단 아래, RMF 입력한 지정값 아래
    매도 조건: VWMA 밴드 상단 위, RMF 입력한 지정값 위
    DCA 접근 방식: 1차, 2차, 3차 분할매수 후 전량매도
    std_dev: 표준 편차 값 (상단 및 하단 밴드의 폭을 결정)
    """
    data['VWMA'] = get_vwma(data)
    data['RMF'] = get_rmf(data)
    data['Upper_Band'] = data['VWMA'] + (std_dev * data['VWMA'].rolling(window=14).std())
    data['Lower_Band'] = data['VWMA'] - (std_dev * data['VWMA'].rolling(window=14).std())

    actions = []
    dca_counter = 0  # DCA 수행 횟수
    for index, row in data.iterrows():
        price = row['Close']
        vwma = row['VWMA']
        rmf = row['RMF']
        rmf_upper = row['Upper_Band']
        rmf_lower = row['Lower_Band']
        if price < rmf_lower and rmf < rmf_threshold:
            if dca_counter < len(dca_levels):
                dca_counter += 1
                actions.append('Buy')
            else:
                actions.append('Stay')
        elif price > rmf_upper and rmf > rmf_threshold:
            dca_counter = 0
            actions.append('Sell')
        else:
            actions.append('Stay')
    
    data['Action'] = actions
    return data

# 코인 설정
ticker = "XRPUSDT"
interval='15m'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000
df = get_ohlcv(client,ticker,interval,limit)

srp_strategy(df, rmf_threshold=30)

,Time,Open,High,Low,Close,Volume,Weighted_Price,VWMA,RMF,Upper_Band,Lower_Band,Action
0,2023-09-08 06:30:00,0.5038,0.5039,0.5024,0.5027,4286625.0,2.154886e+06,NaN,NaN,NaN,NaN,Stay
1,2023-09-08 06:45:00,0.5027,0.5037,0.5024,0.5033,4326750.0,2.177653e+06,NaN,NaN,NaN,NaN,Stay
2,2023-09-08 07:00:00,0.5033,0.5034,0.5021,0.5025,5435110.0,2.731143e+06,NaN,NaN,NaN,NaN,Stay
3,2023-09-08 07:15:00,0.5025,0.5035,0.5023,0.5033,2385196.0,1.200469e+06,NaN,NaN,NaN,NaN,Stay
4,2023-09-08 07:30:00,0.5033,0.5038,0.5030,0.5038,1483522.0,7.473984e+05,NaN,NaN,NaN,NaN,Stay
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-09-18 15:15:00,0.5055,0.5056,0.5038,0.5052,2753116.0,1.390874e+06,0.503028,67.202248,0.506338,0.499718,Stay
996,2023-09-18 15:30:00,0.5052,0.5059,0.5042,0.5053,2938175.0,1.484660e+06,0.503355,70.580903,0.506460,0.500251,Stay
997,2023-09-18 15:45:00,0.5053,0.5057,0.5040,0.5041,2109788.0,1.063544e+06,0.503512,64.594821,0.506355,0.500670,Stay
998,2023-09-18 16:00:00,0.5041,0.5048,0.5026,0.5037,2514572.0,1.266590e+06,0.503629,59.943152,0.506242,0.501017,Stay


In [5]:
df[df['Action']=='Buy']

,Time,Open,High,Low,Close,Volume,Weighted_Price,VWMA,RMF,Upper_Band,Lower_Band,Action
167,2023-09-10 00:15:00,0.5030,0.5031,0.4997,0.5009,6462362.0,3.236997e+06,0.502171,19.603957,0.502932,0.501409,Buy
183,2023-09-10 04:15:00,0.5008,0.5008,0.4954,0.4977,12039143.0,5.991881e+06,0.499701,18.556092,0.500765,0.498636,Buy
226,2023-09-10 15:00:00,0.4974,0.4975,0.4923,0.4954,10129557.0,5.018183e+06,0.497355,17.194699,0.498570,0.496139,Buy
264,2023-09-11 00:30:00,0.4964,0.4965,0.4938,0.4942,3565911.0,1.762273e+06,0.497722,29.336401,0.498775,0.496669,Buy
265,2023-09-11 00:45:00,0.4942,0.4951,0.4924,0.4939,4012078.0,1.981565e+06,0.496849,27.088161,0.498370,0.495329,Buy
290,2023-09-11 07:00:00,0.4924,0.4925,0.4888,0.4890,10732688.0,5.248284e+06,0.493339,19.631688,0.494674,0.492005,Buy
291,2023-09-11 07:15:00,0.4890,0.4899,0.4759,0.4834,40195768.0,1.943063e+07,0.488480,11.518474,0.492372,0.484587,Buy
302,2023-09-11 10:00:00,0.4793,0.4794,0.4735,0.4757,13320435.0,6.336531e+06,0.482615,20.604214,0.489504,0.475725,Buy
330,2023-09-11 17:00:00,0.4716,0.4719,0.4697,0.4701,2454612.0,1.153913e+06,0.472681,29.764622,0.473327,0.472035,Buy
705,2023-09-15 14:45:00,0.4900,0.4908,0.4874,0.4897,7044829.0,3.449853e+06,0.492503,29.708291,0.494681,0.490325,Buy


In [4]:
df[df['Action']=='Sell']

,Time,Open,High,Low,Close,Volume,Weighted_Price,VWMA,RMF,Upper_Band,Lower_Band,Action
29,2023-09-08 13:45:00,0.5007,0.5015,0.5001,0.5008,1922519.0,9.627975e+05,0.499166,55.738916,0.500709,0.497622,Sell
30,2023-09-08 14:00:00,0.5008,0.5015,0.5007,0.5011,1575169.0,7.893172e+05,0.499337,61.244508,0.500702,0.497971,Sell
31,2023-09-08 14:15:00,0.5011,0.5015,0.5006,0.5015,1330042.0,6.670161e+05,0.499643,67.515281,0.500861,0.498426,Sell
34,2023-09-08 15:00:00,0.5003,0.5026,0.5003,0.5017,4616473.0,2.316085e+06,0.500332,63.213863,0.501390,0.499274,Sell
44,2023-09-08 17:30:00,0.5021,0.5037,0.5018,0.5032,1800213.0,9.058672e+05,0.501573,63.789037,0.502822,0.500324,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...
977,2023-09-18 10:45:00,0.4977,0.5001,0.4977,0.4997,4902729.0,2.449894e+06,0.494976,72.168569,0.496189,0.493763,Sell
978,2023-09-18 11:00:00,0.4997,0.5006,0.4984,0.4995,4353614.0,2.174630e+06,0.495824,72.219811,0.497592,0.494056,Sell
979,2023-09-18 11:15:00,0.4994,0.5012,0.4986,0.5005,3725461.0,1.864593e+06,0.496626,77.933012,0.499037,0.494215,Sell
980,2023-09-18 11:30:00,0.5005,0.5016,0.4994,0.5013,3299372.0,1.653975e+06,0.497144,79.082945,0.500127,0.494161,Sell


In [6]:
def srp_strategy(data, rmf_threshold=30, dca_levels=[0.1,0.2,0.5], std_dev=2):
    """
    SRP 전략
    매수 조건: VWMA 밴드 하단 아래, RMF 입력한 지정값 아래
    매도 조건: VWMA 밴드 상단 위, RMF 입력한 지정값 위
    DCA 접근 방식: 1차, 2차, 3차 분할매수 후 전량매도
    std_dev: 표준 편차 값 (상단 및 하단 밴드의 폭을 결정)
    """
    data['VWMA'] = get_vwma(data)
    data['RMF'] = get_rmf(data)
    data['Upper_Band'] = data['VWMA'] + (std_dev * data['VWMA'].rolling(window=14).std())
    data['Lower_Band'] = data['VWMA'] - (std_dev * data['VWMA'].rolling(window=14).std())

    actions = []
    dca_counter = 0  # DCA 수행 횟수
    for index, row in data.iterrows():
        price = row['Close']
        vwma = row['VWMA']
        rmf = row['RMF']
        rmf_upper = row['Upper_Band']
        rmf_lower = row['Lower_Band']
        if price < rmf_lower and rmf < rmf_threshold:
            if dca_counter < len(dca_levels):
                dca_counter += 1
                actions.append('Buy')
            else:
                actions.append('Stay')
        elif price > rmf_upper and rmf > 100-rmf_threshold:
            dca_counter = 0
            actions.append('Sell')
        else:
            actions.append('Stay')
    
    data['Action'] = actions
    return data

# 코인 설정
ticker = "XRPUSDT"
interval='15m'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000
df = get_ohlcv(client,ticker,interval,limit)

srp_strategy(df, rmf_threshold=30)

,Time,Open,High,Low,Close,Volume,Weighted_Price,VWMA,RMF,Upper_Band,Lower_Band,Action
0,2023-09-08 06:30:00,0.5038,0.5039,0.5024,0.5027,4286625.0,2.154886e+06,NaN,NaN,NaN,NaN,Stay
1,2023-09-08 06:45:00,0.5027,0.5037,0.5024,0.5033,4326750.0,2.177653e+06,NaN,NaN,NaN,NaN,Stay
2,2023-09-08 07:00:00,0.5033,0.5034,0.5021,0.5025,5435110.0,2.731143e+06,NaN,NaN,NaN,NaN,Stay
3,2023-09-08 07:15:00,0.5025,0.5035,0.5023,0.5033,2385196.0,1.200469e+06,NaN,NaN,NaN,NaN,Stay
4,2023-09-08 07:30:00,0.5033,0.5038,0.5030,0.5038,1483522.0,7.473984e+05,NaN,NaN,NaN,NaN,Stay
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-09-18 15:15:00,0.5055,0.5056,0.5038,0.5052,2753116.0,1.390874e+06,0.503028,67.202248,0.506338,0.499718,Stay
996,2023-09-18 15:30:00,0.5052,0.5059,0.5042,0.5053,2938175.0,1.484660e+06,0.503355,70.580903,0.506460,0.500251,Stay
997,2023-09-18 15:45:00,0.5053,0.5057,0.5040,0.5041,2109788.0,1.063544e+06,0.503512,64.594821,0.506355,0.500670,Stay
998,2023-09-18 16:00:00,0.5041,0.5048,0.5026,0.5037,2514572.0,1.266590e+06,0.503629,59.943152,0.506242,0.501017,Stay


In [7]:
# 코인 설정
ticker = "ETHUSDT"
interval='15m'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000
df = get_ohlcv(client,ticker,interval,limit)

# 초기 설정
start_balance =100000.0 # 시작 자산
cash = start_balance
fee = 0.0005

# 전략 설정
rmf_threshold=20
dca_levels=[0.2,0.3,0.5,1]
std_dev=2
bt = srp_strategy(df,rmf_threshold,dca_levels,std_dev)

bt[['현금', '보유수량', '총 자산', '수익률']] = 0.0
quantity = 0
dca_level = 0
bt.loc[0, '현금'] = cash
bt.loc[0, '총 자산'] = cash
for i in range(1, len(bt)):
    price = df.loc[i, 'Close']
    if bt.loc[i, 'Action'] == 'Buy' and dca_level < len(dca_levels):
        rate = dca_levels[dca_level]
        buy_quantity = cash*rate / price * (1-fee)  # 매수수량 = 현금*비율 / 현재가*(1-수수료)
        quantity += buy_quantity    # 수량 업데이트
        cash -= cash*rate   # 현금 업데이트
        dca_level += 1
    elif bt.loc[i, 'Action'] == 'Sell' and bt.loc[i, '수익률'] > 0.02:
        cash += quantity * price * (1-fee)
        quantity = 0
        dca_level = 0
    bt.loc[i, '보유수량'] = quantity
    bt.loc[i, '현금'] = cash
    bt.loc[i, '총 자산'] = cash + (quantity*price)
    bt.loc[i, '수익률'] = (bt.loc[i,'총 자산'] - start_balance) / start_balance


print("매수 횟수:", len(bt[bt['Action']=='Buy']), "번")
print("매도 횟수:", len(bt[bt['Action']=='Sell']), "번")
print("최고 수익률:", round(max(bt['수익률']),3), "%")
print("MDD:", round(min(bt['수익률']),3), "%")
print("최종 수익률:", round(bt['수익률'].iloc[-1],3), "%")

bt['수익률']

매수 횟수: 2 번
매도 횟수: 9 번
최고 수익률: 0.024 %
MDD: -0.011 %
최종 수익률: 0.021 %


0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
995    0.020784
996    0.020110
997    0.021441
998    0.021017
999    0.020616
Name: 수익률, Length: 1000, dtype: float64

In [11]:
bt.loc[1, '수익률'] < 0.02

True

In [1]:
import requests
import pandas as pd

# Binance API endpoint URL
url = "https://api.binance.com/api/v1/klines"

# Set the parameters for the API request
params = {
    "symbol": "BTCUSDT",  # Replace with the trading pair you want
    "interval": "15m",    # 15-minute interval
    "limit": 1000,        # Number of candles to fetch per request
}

# List to store data
data_list = []

# Initial request to get the first batch of data
response = requests.get(url, params=params)
data = response.json()
data_list.extend(data)

# Continue making requests until all data is fetched
while len(data) == params["limit"]:
    # Adjust the start time to fetch the next batch
    params["startTime"] = data[-1][0] + 1  # Set the start time to the last timestamp + 1
    response = requests.get(url, params=params)
    data = response.json()
    data_list.extend(data)

# Convert the response data into a DataFrame
df = pd.DataFrame(data_list, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# Select only the relevant columns (OHLCV)
df = df[["timestamp", "open", "high", "low", "close", "volume"]]

# Convert the OHLCV values to numeric
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)

# Print the DataFrame with the OHLCV data
print(df)


              timestamp      open      high       low     close      volume
0   2023-09-08 07:45:00  26240.48  26263.32  26233.99  26256.22   170.21128
1   2023-09-08 08:00:00  26256.21  26266.93  26213.61  26213.61   308.29126
2   2023-09-08 08:15:00  26213.62  26231.07  26196.65  26217.73   250.28654
3   2023-09-08 08:30:00  26217.73  26230.00  26192.25  26202.33   261.60201
4   2023-09-08 08:45:00  26202.33  26212.55  26188.23  26200.04   242.50807
..                  ...       ...       ...       ...       ...         ...
995 2023-09-18 16:30:00  27211.80  27253.42  27203.89  27250.80   529.19386
996 2023-09-18 16:45:00  27250.80  27264.99  27240.26  27246.00   149.18357
997 2023-09-18 17:00:00  27245.99  27268.52  27170.00  27205.65   230.20744
998 2023-09-18 17:15:00  27205.64  27205.65  26900.00  26904.98  1806.75251
999 2023-09-18 17:30:00  26904.98  26948.39  26669.94  26797.00  2658.63668

[1000 rows x 6 columns]


In [2]:
import requests
import pandas as pd
from datetime import datetime

# Binance API endpoint URL
url = "https://api.binance.com/api/v1/klines"

# Set the parameters for the API request
symbol = "BTCUSDT"          # Replace with the trading pair you want
interval = "15m"            # 15-minute interval
start_time = 1609459200000  # Replace with your desired start time in milliseconds (e.g., January 1, 2021)
end_time = 1632921600000    # Replace with your desired end time in milliseconds (e.g., September 30, 2021)

params = {
    "symbol": symbol,
    "interval": interval,
    "limit": 1000,
    "startTime": start_time,
    "endTime": end_time
}

# List to store data
data_list = []

while True:
    response = requests.get(url, params=params)
    data = response.json()
    
    if len(data) == 0:
        break
    
    data_list.extend(data)
    
    # Update the start time for the next request
    params["startTime"] = data[-1][0] + 1

# Convert the response data into a DataFrame
df = pd.DataFrame(data_list, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# Select only the relevant columns (OHLCV)
df = df[["timestamp", "open", "high", "low", "close", "volume"]]

# Convert the OHLCV values to numeric
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)

# Print the DataFrame with the OHLCV data
print(df)


                timestamp      open      high       low     close       volume
0     2021-01-01 00:00:00  28923.63  29017.50  28690.17  28752.80   840.077569
1     2021-01-01 00:15:00  28752.80  28875.55  28720.91  28836.63   480.611179
2     2021-01-01 00:30:00  28836.63  28943.87  28836.62  28930.11   471.134708
3     2021-01-01 00:45:00  28930.11  29031.34  28889.99  28995.13   519.987989
4     2021-01-01 01:00:00  28995.13  29385.00  28960.35  29382.59  1735.839005
...                   ...       ...       ...       ...       ...          ...
26000 2021-09-29 12:15:00  42451.79  42590.00  42135.86  42196.19   703.104040
26001 2021-09-29 12:30:00  42196.18  42265.44  41856.21  42044.44   806.534100
26002 2021-09-29 12:45:00  42044.44  42115.00  41940.98  42050.31   316.138900
26003 2021-09-29 13:00:00  42050.31  42190.00  41989.12  42036.45   469.128200
26004 2021-09-29 13:15:00  42036.44  42109.47  41900.00  42056.71   505.447990

[26005 rows x 6 columns]


In [3]:
import requests
import pandas as pd
from datetime import datetime

# Binance API endpoint URL
url = "https://api.binance.com/api/v1/klines"

# Set the parameters for the API request
symbol = "BTCUSDT"  # Replace with the trading pair you want
interval = "15m"    # 15-minute interval

# Convert start date to milliseconds (2016년 6월 1일)
start_date = datetime(2016, 6, 1)
start_time = int(start_date.timestamp()) * 1000

# Get the current date and time as the end date
end_date = datetime.now()
end_time = int(end_date.timestamp()) * 1000

params = {
    "symbol": symbol,
    "interval": interval,
    "limit": 1000,      # Maximum limit per request
    "startTime": start_time,
    "endTime": end_time
}

# List to store data
data_list = []

while True:
    response = requests.get(url, params=params)
    data = response.json()
    
    if len(data) == 0:
        break
    
    data_list.extend(data)
    
    # Update the start time for the next request
    params["startTime"] = data[-1][0] + 1

# Convert the response data into a DataFrame
df = pd.DataFrame(data_list, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# Select only the relevant columns (OHLCV)
df = df[["timestamp", "open", "high", "low", "close", "volume"]]

# Convert the OHLCV values to numeric
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)

# Print the DataFrame with the OHLCV data
print(df)


                 timestamp      open      high       low     close      volume
0      2017-08-17 04:00:00   4261.48   4280.56   4261.48   4261.48    2.189061
1      2017-08-17 04:15:00   4261.48   4270.41   4261.32   4261.45    9.119865
2      2017-08-17 04:30:00   4280.00   4310.07   4267.99   4310.07   21.923552
3      2017-08-17 04:45:00   4310.07   4313.62   4291.37   4308.83   13.948531
4      2017-08-17 05:00:00   4308.83   4328.69   4304.31   4304.31    5.101153
...                    ...       ...       ...       ...       ...         ...
212883 2023-09-18 14:00:00  27255.22  27279.58  27130.00  27197.27  751.074910
212884 2023-09-18 14:15:00  27197.27  27270.67  27170.62  27238.38  492.969710
212885 2023-09-18 14:30:00  27238.38  27269.93  27217.40  27237.00  429.250000
212886 2023-09-18 14:45:00  27236.99  27292.68  27229.32  27260.46  267.293780
212887 2023-09-18 15:00:00  27260.46  27299.36  27260.46  27298.26  307.726120

[212888 rows x 6 columns]


In [ ]:
from binance.client import Client
from BinanceTrade import *
import pandas as pd
import time

with open('/Users/ho/Documents/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
client = Client(api_key=APP_KEY, api_secret=APP_SECRET)

# 코인 설정
tickers = ["BTCUSDT","ETHUSDT","XRPUSDT","BNBUSDT","ADAUSDT","DOGEUSDT"]
interval='15m'   # 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
limit=1000

while True:
    for ticker in tickers:
        df = pd.read_excel(f"{ticker}/{ticker}{interval}.xlsc")
        

        now = datetime.datetime.now()
        df = get_ohlcv(client, ticker=ticker, limit=limit)
        df.to_excel(f"data/{ticker}/{ticker}{interval}_[{now.strftime('%Y-%m-%d %H:%M')}].xlsx")
        print(f"{now.strftime('%Y-%m-%d %H:%M')} {ticker} 완료!")
    time.sleep(600)

